# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), Dec 5, 2017**

A brief introduction to basic usage of `DataFrames`. Tested under `DataFrames` master on 2017-12-05.
I will try to keep it up to date as the package evolves.

In [1]:
using DataFrames # load package

## Split-apply-combine

In [2]:
x = DataFrame(id=[1,2,3,4,1,2,3,4], id2=[1,2,1,2,1,2,1,2], v=rand(8))

,id,id2,v
1,1,1,0.613302
2,2,2,0.308748
3,3,1,0.427095
4,4,2,0.445219
5,1,1,0.51192
6,2,2,0.0172866
7,3,1,0.580768
8,4,2,0.307158


In [3]:
gx1 = groupby(x, :id) # group by multiple variables

DataFrames.GroupedDataFrame  4 groups with keys: Symbol[:id]
First Group:
2×3 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ id2 │ v        │
├─────┼────┼─────┼──────────┤
│ 1   │ 1  │ 1   │ 0.613302 │
│ 2   │ 1  │ 1   │ 0.51192  │
⋮
Last Group:
2×3 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ id2 │ v        │
├─────┼────┼─────┼──────────┤
│ 1   │ 4  │ 2   │ 0.445219 │
│ 2   │ 4  │ 2   │ 0.307158 │

In [4]:
gx2 = groupby(x, [:id, :id2])

DataFrames.GroupedDataFrame  4 groups with keys: Symbol[:id, :id2]
First Group:
2×3 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ id2 │ v        │
├─────┼────┼─────┼──────────┤
│ 1   │ 1  │ 1   │ 0.613302 │
│ 2   │ 1  │ 1   │ 0.51192  │
⋮
Last Group:
2×3 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ id2 │ v        │
├─────┼────┼─────┼──────────┤
│ 1   │ 4  │ 2   │ 0.445219 │
│ 2   │ 4  │ 2   │ 0.307158 │

In [5]:
vcat(gx2...) # back to the original DataFrame

,id,id2,v
1,1,1,0.613302
2,1,1,0.51192
3,2,2,0.308748
4,2,2,0.0172866
5,3,1,0.427095
6,3,1,0.580768
7,4,2,0.445219
8,4,2,0.307158


In [6]:
x = DataFrame(id = [missing, 5, 1, 3, missing], x = 1:5)

,id,x
1,missing,1
2,5,2
3,1,3
4,3,4
5,missing,5


In [7]:
showall(groupby(x, :id)) # by default groups include mising values and are not sorted

DataFrames.GroupedDataFrame  4 groups with keys: Symbol[:id]
gd[1]:
2×2 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id      │ x │
├─────┼─────────┼───┤
│ 1   │ missing │ 1 │
│ 2   │ missing │ 5 │gd[2]:
1×2 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ x │
├─────┼────┼───┤
│ 1   │ 5  │ 2 │gd[3]:
1×2 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ x │
├─────┼────┼───┤
│ 1   │ 1  │ 3 │gd[4]:
1×2 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ x │
├─────┼────┼───┤
│ 1   │ 3  │ 4 │

In [8]:
showall(groupby(x, :id, sort=true, skipmissing=true)) # but we can change it :)

DataFrames.GroupedDataFrame  3 groups with keys: Symbol[:id]
gd[1]:
1×2 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ x │
├─────┼────┼───┤
│ 1   │ 1  │ 3 │gd[2]:
1×2 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ x │
├─────┼────┼───┤
│ 1   │ 3  │ 4 │gd[3]:
1×2 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ id │ x │
├─────┼────┼───┤
│ 1   │ 5  │ 2 │

In [9]:
x = DataFrame(id=rand('a':'d', 100), v=rand(100));
by(x, :id, y->mean(y[:v])) # apply a function to each group of a data frame

,id,x1
1,'c',0.543879
2,'b',0.566357
3,'d',0.55235
4,'a',0.557881


In [10]:
by(x, :id, y->mean(y[:v]), sort=true) # we can sort the output

,id,x1
1,'a',0.557881
2,'b',0.566357
3,'c',0.543879
4,'d',0.55235


In [11]:
by(x, :id, y->DataFrame(res=mean(y[:v]))) # this way we can set a name for a column - DataFramesMeta @by is better

,id,res
1,'c',0.543879
2,'b',0.566357
3,'d',0.55235
4,'a',0.557881


In [12]:
x = DataFrame(id=rand('a':'d', 100), x1=rand(100), x2=rand(100))
aggregate(x, :id, sum) # apply a function over all columns of a data frame in groups given by id

,id,x1_sum,x2_sum
1,'b',10.7377,10.8649
2,'c',10.5219,10.6734
3,'a',15.9332,18.5059
4,'d',11.2631,9.41057


In [13]:
aggregate(x, :id, sum, sort=true) # also can be sorted

,id,x1_sum,x2_sum
1,'a',15.9332,18.5059
2,'b',10.7377,10.8649
3,'c',10.5219,10.6734
4,'d',11.2631,9.41057


*We omit the discussion of of map/combine as I do not find them very useful (better to use by)*

In [14]:
x = DataFrame(rand(3, 5))

,x1,x2,x3,x4,x5
1,0.834378,0.455071,0.0343202,0.886276,0.494359
2,0.805254,0.299038,0.788217,0.176212,0.195506
3,0.605343,0.946998,0.630307,0.817551,0.295498


In [15]:
map(mean, eachcol(x)) # map a function over each column and return a data frame

,x1,x2,x3,x4,x5
1,0.748325,0.567036,0.484281,0.62668,0.328454


In [16]:
foreach(c -> println(c[1], ": ", mean(c[2])), eachcol(x)) # a raw iteration returns a tuple with column name and values

x1: 0.7483250471393426
x2: 0.5670357789219573
x3: 0.48428126564113816
x4: 0.6266796255392941
x5: 0.3284543511272236


In [17]:
colwise(mean, x) # colwise is similar, but produces a vector

5-element Array{Float64,1}:
 0.748325
 0.567036
 0.484281
 0.62668 
 0.328454

In [18]:
x[:id] = [1,1,2]
colwise(mean,groupby(x, :id)) # and works on GroupedDataFrame

2-element Array{Array{Float64,1},1}:
 [0.819816, 0.377055, 0.411269, 0.531244, 0.344933, 1.0]
 [0.605343, 0.946998, 0.630307, 0.817551, 0.295498, 2.0]

In [19]:
map(r -> r[:x1]/r[:x2], eachrow(x)) # now the returned value is DataFrameRow which works similarly to a one-row DataFrame

3-element Array{Float64,1}:
 1.83351 
 2.69281 
 0.639223